In [ ]:
#For 3 colored 3-graphs without mono edge
#I think this is the unordered, so kinda useless

def _identifyT(n, ftype_points, edges, A, B):
    g = Graph([list(range(n+len(edges)+2)), [(i+n,x) for i,b in enumerate(edges) for x in b] + 
               [(a[0], n+len(edges)) for a in A] + 
               [(b[0], n+len(edges)+1) for b in B]], 
              format='vertices_and_edges')
    partt = [[ii] for ii in ftype_points] + \
            [[ii for ii in range(n) if ii not in ftype_points]] + \
            [list(range(n,n+len(edges)))] + [[n+len(edges)], [n+len(edges)+1]]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

def _generateT(n):
    ThreeGraphTheory.exclude(ThreeGraphTheory(4))
    for xx in ThreeGraphTheory.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        for yy in itertools.product(range(3), repeat=int(n)):
            good = True
            for ee in itertools.combinations(range(n), 3):
                if list(ee) in edges:
                    continue
                elif len(set([yy[ii] for ii in ee]))==1:
                    good = False
                    break
            if good:
                A = [[ii] for ii, oo in enumerate(yy) if oo==1]
                B = [[ii] for ii, oo in enumerate(yy) if oo==2]
                iden = _identifyT(n, [], edges, A, B)
                if iden not in unique:
                    unique.append(iden)
                    yield {'edges': edges, 'A': A, 'B': B}

T = CombinatorialTheory('Ord3GNoMonoNoK4', _generateT, _identifyT, edges=3, A=1, B=1)

In [ ]:
#should be 240409 (too large)
len(T.generate_flags(6))

In [ ]:
#This is example code to create colored theories

def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
    is_graph = (k==2)
    color_number = sum(len(xx) for xx in order_partition)
    edges = kwargs["edges"]
    Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
    g_parts = [[ii] for ii in ftype_points] + \
              [[ii for ii in range(n) if ii not in ftype_points]]
    ppadd = 0 if is_graph else len(edges)
    g_verts = list(range(n+ppadd+color_number))
    g_parts.append(list(range(n, n+ppadd)))

    g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
    if is_graph:
        g_edges = list(edges)
        for ii in range(color_number):
            g_edges += [(xx, n+ii) for xx in Cs[ii]]
    else:
        g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
        for ii in range(color_number):
            g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
    g = Graph([g_verts, g_edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

def _generateCT(k, order_partition, n):
    color_number = sum(len(xx) for xx in order_partition)
    if k==2:
        BT = GraphTheory
    if k==3:
        BT = ThreeGraphTheory
    for xx in BT.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        
        for yy in itertools.product(range(color_number), repeat=int(n)):
            yy = list(yy)
            Cs = {"C{}".format(cc):[[ii] for ii, oo in enumerate(yy) if oo==cc] for cc in range(color_number)}
            iden = _identifyCT(k==2, order_partition, n, [], edges=edges, **Cs)
            if iden not in unique:
                unique.append(iden)
                Cs["edges"] = edges
                yield Cs

In [ ]:
#Idk where the code is to create the unordered theory, so here is a quick guess

#this does the complement (so empty 4 is excluded)

def _identifyT(n, ftype_points, edges, A, B, C):
    g = Graph([list(range(n+len(edges)+3)), [(i+n,x) for i,b in enumerate(edges) for x in b] + 
               [(a[0], n+len(edges)) for a in A] + 
               [(b[0], n+len(edges)+1) for b in B] + 
               [(c[0], n+len(edges)+2) for c in C]], 
              format='vertices_and_edges')
    partt = [[ii] for ii in ftype_points] + \
            [[ii for ii in range(n) if ii not in ftype_points]] + \
            [list(range(n,n+len(edges)))] + [[n+len(edges), n+len(edges)+1, n+len(edges)+2]]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

def _generateT(n):
    ThreeGraphTheory.exclude(ThreeGraphTheory(4))
    for xx in ThreeGraphTheory.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        for yy in itertools.product(range(3), repeat=int(n-1)):
            good = True
            yy = list(yy) + [0]
            for ee in itertools.combinations(range(n), 3):
                if list(ee) in edges:
                    continue
                elif len(set([yy[ii] for ii in ee]))==1:
                    good = False
                    break
            if good:
                A = [[ii] for ii, oo in enumerate(yy) if oo==1]
                B = [[ii] for ii, oo in enumerate(yy) if oo==2]
                C = [[ii] for ii, oo in enumerate(yy) if oo==3]
                iden = _identifyT(n, [], edges, A, B, C)
                if iden not in unique:
                    unique.append(iden)
                    yield {'edges': edges, 'A': A, 'B': B, 'C':C}

TT = CombinatorialTheory('UnOrd3GNoMonoNoK4', _generateT, _identifyT, edges=3, A=1, B=1, C=1)

In [ ]:
target_element = TT(3)
#I forgot what was here, probably degree equality, edge density lower bound, stuff like that
poss = []
res = TT.optimize_problem(target_element, 5, maximize=True, certificate=False, positives=poss)